# pothole 탐지모델
# 1. 모델 YOLOV5 다운로드

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt # install dependencies

import torch
from IPython.display import Image, clear_output
# display=utils.notebook_init()

# 2. 추론(Inference)

### detect.py- yolov5 모델을 이용한 추론

### 주어진 yolov5/data/images의 이미지를 바운딩 박스 작업(detect)

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images/

-  detect한 이미지를 출력해보자.
- 위 1에서 "# display=utils.notebook_init()" 했으므로, 화면출력시 display 사용하지 못함

In [ ]:
Image(filename = '/content/yolov5/runs/detect/exp/bus.jpg', width=600)

# 3. 포트홀 데이터셋 다운로드 하기

### 포트홀 데이터셋 다운로드(roboflow)

포트홀 데이터셋: https://public.roboflow.com/object-detection/pothole

## 포트홀 데이터셋을 설치한다.

In [ ]:
%mkdir /content/pothole
%cd /content/pothole
!curl -L "https://public.roboflow.com/ds/m0E5Tx4om4?key=kEKT9jhfJu" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

# 4. data.yaml을 열어서 수정하지 않고, 코딩으로 yaml 내용을 수정하기

## list를 정리(train, test, valid의 이미지 갯수 확인)

In [ ]:
from glob import glob

train_img_list=glob('/content/pothole/train/images/*.jpg')
test_img_list=glob('/content/pothole/test/images/*.jpg')
valid_img_list=glob('/content/pothole/valid/images/*.jpg')
print(len(train_img_list),len(test_img_list), len(valid_img_list))

## txt화일로 저장

In [ ]:
import yaml

with open('/content/pothole/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')
with open('/content/pothole/test.txt', 'w') as f:
  f.write('\n'.join(test_img_list) + '\n')  
with open('/content/pothole/val.txt', 'w') as f:
  f.write('\n'.join(valid_img_list) + '\n')

## 현재 data.yaml화일을 열어보자.

In [ ]:
%cat /content/pothole/data.yaml

## yaml 의 내용을 수정할 수 있도록 writetemplate 명령을 사용할 수 있게 수정한다.

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
  with open(line, 'w') as f:
    f.write(cell.format(**globals()))

## 내용을 수정한다.

In [ ]:
%%writetemplate /content/pothole/data.yaml

train: /content/pothole/train/images
test: /content/pothole/test/images
val: /content/pothole/valid/images

nc: 1
names: ['pothole']

## 원하는데로 저장이 되는지 확인한다.

In [ ]:
%cat /content/pothole/data.yaml

# 5. pothole 학습(train)하기

## - train.py 적용

- cfg : 학습에 사용할 모델의 경로설정(yolov5s 등이 있는 곳의 경로)

In [ ]:
%%time
%cd /content/yolov5
!python train.py --img 640 --batch 16 --epochs 10 --data /content/pothole/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name pothole_results --cache



# 6. 학습 결과로 얻은 모델(last.pt)로 potholt/train/images 화일 detect 하기

- detect.py 적용

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/pothole_results/weights/last.pt --img 640 --conf 0.25 --source /content/pothole/train/images

- detect한 이미지 화일을 출력하기

In [ ]:
Image(filename = '/content/yolov5/runs/detect/exp2/img-100_jpg.rf.233751cac3399aa80b5e58a2afbb753a.jpg', width=1000)

# 7. 셀 실행할 때 마다, 랜덤으로 이미지 출력하기

- 처리하지 않은  pothole/test/images 이미지 출력하기

In [ ]:
import glob
import random
from IPython.display import Image, display

Image_name = random.choice(glob.glob('/content/pothole/test/images/*.jpg'))
display(Image(filename=Image_name))

- detect한 이미지 출력하기

In [ ]:
import glob
import random
from IPython.display import Image, display

Image_name = random.choice(glob.glob('/content/yolov5/runs/detect/exp2/*.jpg'))
display(Image(filename=Image_name))

# 8. 모델을 구글 드라이브로 출력
구글 코랩에서 드라이브를 불러온다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 구글 드라이브에 pothole 디렉토리를 만들고, best.pt 값을 copy한다.

In [ ]:
%mkdir /content/drive/My\ Drive/pothole
%cp /content/yolov5/runs/train/pothole_results/weights/best.pt /content/drive/My\ Drive/pothole/